In [1]:
import time as t
import datetime as dt
import pandas as pd
from telethon import TelegramClient
import os
import sys
sys.path.insert(1, '../')
from utils.common import *

telegram_api_id = os.getenv('telegram_api_id')
telegram_api_hash = os.getenv('telegram_api_hash')
telegram_phone_number = os.getenv('telegram_phone_number')
pass2fa = os.getenv('telegram_pass2fa')

In [2]:
client = TelegramClient(session=None,
                        api_id=telegram_api_id,
                        auto_reconnect=True,
                        api_hash=telegram_api_hash)

In [3]:
await client.start(phone=telegram_phone_number, password=pass2fa)

Signed in successfully as Jack Ma; remember to not break the ToS or you will risk an account ban!


In [4]:
async for dialog in client.iter_dialogs():
    print(dialog.name, 'has ID', dialog.id)

Solana Scanner has ID -1002023951506
Telegram has ID 777000
Solana New Liquidity Pools has ID -1002039712427
Gambles 🎲 MadApes has ID -1001758611100
Degen Seals has ID -1001523523939
Marat Gudkov has ID 178008364
Jack Ma has ID 1223950100
Yield Board has ID -1001628892345


In [5]:
lp_chat = -1002039712427

In [6]:
## read last message
message =  await client.get_messages(lp_chat, limit=5)
message = pd.DataFrame({'messages':message})
message['id'] = message['messages'].apply(lambda x:x.id)
message['date'] = message['messages'].apply(lambda x:x.date)
message['text'] = message['messages'].apply(lambda x:x.text)
message['date'] = pd.to_datetime(message['date'])
message['date'] = message['date'].dt.tz_localize(None)

pattern = r'`([1-9A-HJ-NP-Za-km-z]{32,44})`'
message['address'] = message['text'].str.extract(pattern)
message['expected_launch_time'] = message['text'].str.extract(r'(?<=Launch:\*\* )(.*)')[0].str.strip('`')
message['mcap'] = message['text'].str.extract(r'(?<=Mcap:\*\* )(.*)')[0].str.strip('`')
message['liq'] = message['text'].str.extract(r'(?<=Liq:\*\* )(.*)')[0].str.strip('`')
message['s_mm'] = message['text'].str.extract(r'(?<=Mutable Metadata: )(.*)')
message['s_ma'] = message['text'].str.extract(r'(?<=Mint Authority: )(.*)')
message['s_fa'] = message['text'].str.extract(r'(?<=Freeze Authority: )(.*)')
message['s_s'] = message['text'].str.extract(r'(?<=Score: )(.*)')

In [7]:
## Parse
message.loc[message['s_mm'].str.contains('Yes'), 's_mm2'] = True
message.loc[message['s_mm'].str.contains('No'), 's_mm2'] = False
message.loc[message['s_ma'].str.contains('Yes'), 's_ma2'] = True
message.loc[message['s_ma'].str.contains('No'), 's_ma2'] = False
message.loc[message['s_fa'].str.contains('Yes'), 's_fa2'] = True
message.loc[message['s_fa'].str.contains('No'), 's_fa2'] = False
message.loc[message['s_s'].str.contains('Bad'), 's_q'] = 0
message.loc[message['s_s'].str.contains('Neutral'), 's_q'] = 1
message.loc[message['s_s'].str.contains('Good'), 's_q'] = 2
message['s_sni'] = message['s_s'].str.extract(r'([0-9]+)')
message['s_sni'] = message['s_sni'].fillna(0)

mult = {'K':1e3,'M':1e6,'B':1e9,'T':1e12}
message['mcap'] = message['mcap'].str.strip('[$\*]')
message['mcap_num'] = message['mcap'].str.extract('([0-9\.]+)').astype(float) * message['mcap'].str.extract('([A-Z])').replace(mult).fillna(1)
message['liq'] = message['liq'].str.strip(r" \[.*\]")
message['liq_num'] = message['liq'].str.extract('([0-9\.]+)').astype(float) * message['liq'].str.extract('([A-Z])').replace(mult).fillna(1)
message[['s_q','mcap_num','liq_num']] = message[['s_q','mcap_num','liq_num']].astype(int)

In [8]:
## get start time
tp = message['expected_launch_time'].str.extractall('([0-9]+)').unstack()
tp.columns = tp.columns.droplevel(0)
tp['hours'] = message['expected_launch_time'].str.contains('hour')
tp['sec'] = message['expected_launch_time'].str.contains('second')
tp.loc[tp['hours']==True, 'new'] = tp.loc[tp['hours']==True].apply(lambda x: dt.timedelta(hours=int(x[0]), minutes=int(x[1])), axis=1)
tp.loc[tp['sec']==True, 'new'] = tp.loc[tp['sec']==True].apply(lambda x: dt.timedelta(seconds=int(x[0])), axis=1)
tp.loc[tp['new'].isna(), 'new'] = tp.loc[tp['new'].isna()].apply(lambda x: dt.timedelta(minutes=int(x[0])), axis=1)
tp.loc[tp['new']>=dt.timedelta(days=30),'new'] = dt.timedelta(seconds=0)

message = pd.concat([message, tp[['new']]],axis=1)
message.loc[(message['expected_launch_time'].str.contains('ago')), 'expected_launch_time_ts'] = message['date'] - pd.to_timedelta(message['new'])
message.loc[(message['expected_launch_time'].str.contains('In')), 'expected_launch_time_ts'] = message['date'] + pd.to_timedelta(message['new'])


In [9]:
message = message.drop(columns=['messages','s_mm','s_ma','s_fa','s_s','mcap','liq','text','new'])

In [10]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT DISTINCT address FROM calls" 
    tracked_a = pd.read_sql_query(query, conn)
    message_clean = message.loc[~message['address'].isin(tracked_a['address'].unique())]
    message_clean.to_sql('calls', conn, if_exists='append', index=False)

In [2]:
## check
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT * FROM calls"
    df = pd.read_sql_query(query, conn)
df

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,215618,2024-04-18 14:35:57,CfQSSRk6Md2cnZCzkJu2QjAVua5FzGMCua7CrhQGRURK,In 1 minute,2024-04-18 14:36:57,0,0,0,2,0,9050,18110,1,2024-04-18 14:37:00,None,None,None
1,215619,2024-04-18 14:37:11,GKMRt3xZznGK4wxoBw3SZfMTwVR9LLFTJxbqMCUvMabg,11 seconds ago,2024-04-18 14:37:00,1,0,0,0,2,1390,2500,1,2024-04-18 14:37:00,None,None,None
2,215620,2024-04-18 14:37:58,66Yp2kKFLsEjszAMVttwHFDZRZRyGt9kbKoXnuv99Dx3,36 seconds ago,2024-04-18 14:37:22,1,0,0,0,4,1730,1390,1,2024-04-18 14:42:00,None,None,None
3,215621,2024-04-18 14:38:20,Ecz4mak5jScXrwZkMGJB3mqqpeuJPWPTddJW6Rgmt9Za,27 seconds ago,2024-04-18 14:37:53,0,0,0,0,2,1110,1110,1,2024-04-18 14:38:00,None,None,None
4,215622,2024-04-18 14:39:15,BdSSmAmWAE8F2yhjAPW91Boa4fQfu98EKt3xxgLKQVxn,36 seconds ago,2024-04-18 14:38:39,1,1,0,0,2,40230,80460,1,2024-04-18 14:39:00,None,None,None
5,218927,2024-04-21 01:14:29,HbzH8fFYQRLjHgDK7G7qXYzWdV4fnwTSdb71zAZvJyDS,14 seconds ago,2024-04-21 01:14:15,0,0,0,2,0,1650,3290,1,2024-04-21 01:14:00+00:00,None,None,None
6,218928,2024-04-21 01:14:51,6xPz9dztCA6TdMsPZ6AHAZ6NEdGMjrouLb3fov8vbBJF,476017 hours and 14 minutes ago,2024-04-21 01:14:51,0,0,0,0,1,770,1500,1,2024-04-21 01:14:00,None,None,None
7,218929,2024-04-21 01:16:28,6XejiuxHwr31q7iZr4bYWzuNzzKoQWh4X4rrYP1qkt6t,13 seconds ago,2024-04-21 01:16:15,1,1,0,0,3,1490,2990,1,2024-04-21 01:16:00,None,None,None
8,218930,2024-04-21 01:17:25,7VEqMtsR2bRFU3g5cMTHuCYHdR6x1cwoVfoQ1uenJVqs,476017 hours and 17 minutes ago,2024-04-21 01:17:25,0,0,0,2,0,57080,23620,1,2024-04-21 01:17:00,None,None,None
9,218931,2024-04-21 01:17:26,9irAyMcsQSemhFiiSCZi6gFyQVsSWCKZgQT4MFd3ocdV,476017 hours and 17 minutes ago,2024-04-21 01:17:26,0,0,0,2,0,57080,23620,1,2024-04-21 01:17:00,None,None,None
